# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [50]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet'])
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterResp.db')
df = pd.read_sql_table('message', con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = df['message']
Y = df.loc[:, 'related':'direct_report']

### 2. Write a tokenization function to process your text data

In [16]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    '''
    Tokenize text data for modeling process.
    It takes as entry text representing messages
    and return as output cleaned and tokenized text
    '''
    # Detect URL
    detected_urls = re.findall(url_regex, text) 
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    # Tokenize text data
    token = word_tokenize(text)
    # Lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Proceed cleaning tokens
    clean_token = []
    for tokn in token:
        clean_tokn = lemmatizer.lemmatize(tokn).lower().strip()
        clean_token.append(clean_tokn)
    return clean_token

###### Testing function to detect any errors before proceeding with machine learning phase

In [19]:
text = "Weather update - a cold front from Cuba this morning"
tokenize(text)

['weather',
 'update',
 '-',
 'a',
 'cold',
 'front',
 'from',
 'cuba',
 'this',
 'morning']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [22]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [32]:
Y_pred = pipeline.predict(X_test)
category_name = Y_test.columns.tolist()
Y_pred_dtf = pd.DataFrame(Y_pred, columns = category_name)
Y_pred_dtf.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# Classification Report
for i in range(36):
    print(category_name[i],\
          '\n',\
          classification_report(Y_test.iloc[:,i], Y_pred_dtf.iloc[:,i]))

related 
              precision    recall  f1-score   support

          0       0.60      0.35      0.44      1537
          1       0.82      0.93      0.87      4960
          2       0.69      0.16      0.26        57

avg / total       0.77      0.79      0.76      6554

request 
              precision    recall  f1-score   support

          0       0.88      0.99      0.93      5426
          1       0.84      0.36      0.50      1128

avg / total       0.87      0.88      0.86      6554

offer 
              precision    recall  f1-score   support

          0       0.99      1.00      1.00      6517
          1       0.00      0.00      0.00        37

avg / total       0.99      0.99      0.99      6554

aid_related 
              precision    recall  f1-score   support

          0       0.72      0.88      0.79      3829
          1       0.75      0.52      0.62      2725

avg / total       0.73      0.73      0.72      6554

medical_help 
              precision    reca

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [40]:
#Accuracy
for i in range(36):
    category = category_name[i]
    acc = accuracy_score(Y_test.iloc[:,i], Y_pred_dtf.iloc[:,i])
    prec = precision_score(Y_test.iloc[:,i], Y_pred_dtf.iloc[:,i], average='micro')
    recal = recall_score(Y_test.iloc[:,i], Y_pred_dtf.iloc[:,i], average='micro')
    f1 = f1_score(Y_test.iloc[:,i], Y_pred_dtf.iloc[:,i], average='micro')
    print(category)
    print("\t Accuracy: %.4f \t Precision: %.4f \t Recall: %.4f \t F1-Score: %.4f \n" %\
              (acc, prec, recal, f1))

related
	 Accuracy: 0.7868 	 Precision: 0.7868 	 Recall: 0.7868 	 F1-Score: 0.7868 

request
	 Accuracy: 0.8781 	 Precision: 0.8781 	 Recall: 0.8781 	 F1-Score: 0.8781 

offer
	 Accuracy: 0.9944 	 Precision: 0.9944 	 Recall: 0.9944 	 F1-Score: 0.9944 

aid_related
	 Accuracy: 0.7295 	 Precision: 0.7295 	 Recall: 0.7295 	 F1-Score: 0.7295 

medical_help
	 Accuracy: 0.9171 	 Precision: 0.9171 	 Recall: 0.9171 	 F1-Score: 0.9171 

medical_products
	 Accuracy: 0.9498 	 Precision: 0.9498 	 Recall: 0.9498 	 F1-Score: 0.9498 

search_and_rescue
	 Accuracy: 0.9745 	 Precision: 0.9745 	 Recall: 0.9745 	 F1-Score: 0.9745 

security
	 Accuracy: 0.9809 	 Precision: 0.9809 	 Recall: 0.9809 	 F1-Score: 0.9809 

military
	 Accuracy: 0.9693 	 Precision: 0.9693 	 Recall: 0.9693 	 F1-Score: 0.9693 

child_alone
	 Accuracy: 1.0000 	 Precision: 1.0000 	 Recall: 1.0000 	 F1-Score: 1.0000 

water
	 Accuracy: 0.9463 	 Precision: 0.9463 	 Recall: 0.9463 	 F1-Score: 0.9463 

food
	 Accuracy: 0.9161 	 Precision

### 6. Improve your model
Use grid search to find better parameters. 

In [41]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f362052a048>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [42]:
# GridSearchCV to imporve model
parameters = {'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__min_samples_leaf':[2, 5, 10]}
cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [43]:
cv.fit(X_train.as_matrix(), Y_train.as_matrix()) 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__max_depth': [10, 50, None], 'clf__estimator__min_samples_leaf': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [45]:
# predict and create a dataframe
Y_prd = cv.predict(X_test)
Y_prd_df = pd.DataFrame(Y_prd, columns = category_name)
for i in range(36):
    category = category_name[i]
    acc = accuracy_score(Y_test.iloc[:,i], Y_prd_df.iloc[:,i])
    prec = precision_score(Y_test.iloc[:,i], Y_prd_df.iloc[:,i], average='micro')
    recal = recall_score(Y_test.iloc[:,i], Y_prd_df.iloc[:,i], average='micro')
    f1 = f1_score(Y_test.iloc[:,i], Y_prd_df.iloc[:,i], average='micro')
    print(category)
    print("\t Accuracy: %.4f \t Precision: %.4f \t Recall: %.4f \t F1-Score: %.4f \n" %\
              (acc, prec, recal, f1))

related
	 Accuracy: 0.7836 	 Precision: 0.7836 	 Recall: 0.7836 	 F1-Score: 0.7836 

request
	 Accuracy: 0.8776 	 Precision: 0.8776 	 Recall: 0.8776 	 F1-Score: 0.8776 

offer
	 Accuracy: 0.9944 	 Precision: 0.9944 	 Recall: 0.9944 	 F1-Score: 0.9944 

aid_related
	 Accuracy: 0.7553 	 Precision: 0.7553 	 Recall: 0.7553 	 F1-Score: 0.7553 

medical_help
	 Accuracy: 0.9187 	 Precision: 0.9187 	 Recall: 0.9187 	 F1-Score: 0.9187 

medical_products
	 Accuracy: 0.9464 	 Precision: 0.9464 	 Recall: 0.9464 	 F1-Score: 0.9464 

search_and_rescue
	 Accuracy: 0.9727 	 Precision: 0.9727 	 Recall: 0.9727 	 F1-Score: 0.9727 

security
	 Accuracy: 0.9814 	 Precision: 0.9814 	 Recall: 0.9814 	 F1-Score: 0.9814 

military
	 Accuracy: 0.9698 	 Precision: 0.9698 	 Recall: 0.9698 	 F1-Score: 0.9698 

child_alone
	 Accuracy: 1.0000 	 Precision: 1.0000 	 Recall: 1.0000 	 F1-Score: 1.0000 

water
	 Accuracy: 0.9387 	 Precision: 0.9387 	 Recall: 0.9387 	 F1-Score: 0.9387 

food
	 Accuracy: 0.9072 	 Precision

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [47]:
pipeline = Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
                    ])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

pipeline.fit(X_train.as_matrix(), Y_train.as_matrix())

# predict and create a dataframe
Y_prd = pipeline.predict(X_test)

# save the improved model in to a data frame
Y_prd_imp = pd.DataFrame(Y_prd, columns = category_name)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


In [48]:
for i in range(36):
    category = category_name[i]
    acc = accuracy_score(Y_test.iloc[:,i], Y_prd_imp.iloc[:,i])
    prec = precision_score(Y_test.iloc[:,i], Y_prd_imp.iloc[:,i], average='micro')
    recal = recall_score(Y_test.iloc[:,i], Y_prd_imp.iloc[:,i], average='micro')
    f1 = f1_score(Y_test.iloc[:,i], Y_prd_imp.iloc[:,i], average='micro')
    print(category)
    print("\t Accuracy: %.4f \t Precision: %.4f \t Recall: %.4f \t F1-Score: %.4f \n" %\
              (acc, prec, recal, f1))

related
	 Accuracy: 0.7370 	 Precision: 0.7370 	 Recall: 0.7370 	 F1-Score: 0.7370 

request
	 Accuracy: 0.8467 	 Precision: 0.8467 	 Recall: 0.8467 	 F1-Score: 0.8467 

offer
	 Accuracy: 0.9907 	 Precision: 0.9907 	 Recall: 0.9907 	 F1-Score: 0.9907 

aid_related
	 Accuracy: 0.6984 	 Precision: 0.6984 	 Recall: 0.6984 	 F1-Score: 0.6984 

medical_help
	 Accuracy: 0.8972 	 Precision: 0.8972 	 Recall: 0.8972 	 F1-Score: 0.8972 

medical_products
	 Accuracy: 0.9391 	 Precision: 0.9391 	 Recall: 0.9391 	 F1-Score: 0.9391 

search_and_rescue
	 Accuracy: 0.9564 	 Precision: 0.9564 	 Recall: 0.9564 	 F1-Score: 0.9564 

security
	 Accuracy: 0.9693 	 Precision: 0.9693 	 Recall: 0.9693 	 F1-Score: 0.9693 

military
	 Accuracy: 0.9614 	 Precision: 0.9614 	 Recall: 0.9614 	 F1-Score: 0.9614 

child_alone
	 Accuracy: 1.0000 	 Precision: 1.0000 	 Recall: 1.0000 	 F1-Score: 1.0000 

water
	 Accuracy: 0.9580 	 Precision: 0.9580 	 Recall: 0.9580 	 F1-Score: 0.9580 

food
	 Accuracy: 0.9426 	 Precision

### 9. Export your model as a pickle file

In [51]:
pickle.dump(cv, open('model_classf.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.